# New section

In [ ]:
!pip install openpyxl beautifulsoup4 requests

Mounted at /gdrive
[Errno 2] No such file or directory: '/gdrive/MyDrive/project/Data_Extraction_and_NLP/TestAssignment'
/content


In [ ]:
from google.colab import drive
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
import re

drive.mount('/content/drive')
%cd '/content/drive/MyDrive/20211030 Test Assignment'

# reading the url files
df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')

# traverse in loop over the df
for index, row in df.iterrows():
    url = row['URL']
    url_id = row['URL_ID']

    #requesting the url
    header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}
    try:
        response = requests.get(url, headers=header)
    except:
        print("can't get response of {}".format(url_id))
        continue

    #creating a beautifulsoup object
    try:
        soup = BeautifulSoup(response.content, 'html.parser')
    except:
        print("can't get page of {}".format(url_id))
        continue

    #find title from the texts
    try:
        title = soup.find('h1').get_text()
    except:
        print("can't get title of {}".format(url_id))
        continue

    # find texts
    article = ""
    try:
        for p in soup.find_all('p'):
            article += p.get_text()
    except:
        print("can't get text of {}".format(url_id))

    # write title and text to the file
    file_name = '/content/drive/MyDrive/20211030 Test Assignment' + str(url_id) + '.txt'
    with open(file_name, 'w') as file:
        file.write(title + '\n' + article)

# Directories
text_dir = "/content/drive/MyDrive/20211030 Test Assignment"
stopwords_dir = "/content/drive/MyDrive/20211030 Test Assignment/StopWords"
sentiment_dir = "/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary"

# loading all stop words from the stopwords directory and storing in the set variable
stop_words = set()
for files in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, files), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(set(f.read().splitlines()))

# loading  all text files from the directory and storing in a list
docs = []
for text_file in os.listdir(text_dir):
    with open(os.path.join(text_dir, text_file), 'r') as f:
        text = f.read()
        # tokenizing the text file
        words = word_tokenize(text)
        # remove the stop words from the tokens
        filtered_text = [word for word in words if word.lower() not in stop_words]
        # add each filtered tokens of each file into a list
        docs.append(filtered_text)

# store positive, negative words from the directory
pos = set()
neg = set()

for files in os.listdir(sentiment_dir):
    if files == 'positive-words.txt':
        with open(os.path.join(sentiment_dir, files), 'r', encoding='ISO-8859-1') as f:
            pos.update(f.read().splitlines())
    else:
        with open(os.path.join(sentiment_dir, files), 'r', encoding='ISO-8859-1') as f:
            neg.update(f.read().splitlines())

# now collect the positive and negative words from each file
# calculate the scores from the positive and negative words
positive_words = []
Negative_words = []
positive_score = []
negative_score = []
polarity_score = []
subjectivity_score = []

# iterate through the list of docs
for i in range(len(docs)):
    positive_words.append([word for word in docs[i] if word.lower() in pos])
    Negative_words.append([word for word in docs[i] if word.lower() in neg])
    positive_score.append(len(positive_words[i]))
    negative_score.append(len(Negative_words[i]))
    polarity_score.append((positive_score[i] - negative_score[i]) / ((positive_score[i] + negative_score[i]) + 0.000001))
    subjectivity_score.append((positive_score[i] + negative_score[i]) / ((len(docs[i])) + 0.000001))

# Average Sentence Length = the number of words / the number of sentences
# Percentage of Complex words = the number of complex words / the number of words
# Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)

avg_sentence_length = []
Percentage_of_Complex_words = []
Fog_Index = []
complex_word_count = []
avg_syllable_word_count = []

stopwords = set(stopwords.words('english'))

def measure(file):
    with open(os.path.join(text_dir, file), 'r') as f:
        text = f.read()
        # remove punctuations
        text = re.sub(r'[^\w\s.]', '', text)
        # split the given text file into sentences
        sentences = text.split('.')
        # total number of sentences in a file
        num_sentences = len(sentences)
        # total words in the file
        words = [word for word in text.split() if word.lower() not in stopwords]
        num_words = len(words)

        # complex words having syllable count is greater than 2
        # Complex words are words in the text that contain more than two syllables.
        complex_words = []
        for word in words:
            vowels = 'aeiou'
            syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
            if syllable_count_word > 2:
                complex_words.append(word)

        # Syllable Count Per Word
        # We count the number of Syllables in each word of the text by counting the vowels present in each word.
        # We also handle some exceptions like words ending with "es","ed" by not counting them as a syllable.
        syllable_count = 0
        syllable_words = []
        for word in words:
            if word.endswith('es'):
                word = word[:-2]
            elif word.endswith('ed'):
                word = word[:-2]
            vowels = 'aeiou'
            syllable_count_word = sum(1 for letter in word if letter.lower() in vowels)
            if syllable_count_word >= 1:
                syllable_words.append(word)
                syllable_count += syllable_count_word

        avg_sentence_len = num_words / num_sentences
        avg_syllable_word_count = syllable_count / len(syllable_words)
        Percent_Complex_words = len(complex_words) / num_words
        Fog_Index = 0.4 * (avg_sentence_len + Percent_Complex_words)

        return avg_sentence_len, Percent_Complex_words, Fog_Index, len(complex_words), avg_syllable_word_count

# iterate through each file or doc
for file in os.listdir(text_dir):
    x, y, z, a, b = measure(file)
    avg_sentence_length.append(x)
    Percentage_of_Complex_words.append(y)
    Fog_Index.append(z)
    complex_word_count.append(a)
    avg_syllable_word_count.append(b)

# Word Count and Average Word Length Sum of the total number of characters in each word/Total number of words
# We count the total cleaned words present in the text by
# removing the stop words (using stopwords class of nltk package).
# removing any punctuations like ? ! , . from the word before counting.

def cleaned_words(file):
    with open(os.path.join(text_dir, file), 'r') as f:
        text = f.read()
        text = re.sub(r'[^\w\s]', '', text)
        words = [word for word in text.split() if word.lower() not in stopwords]
        length = sum(len(word) for word in words)
        average_word_length = length / len(words)
    return len(words), average_word_length

word_count = []
average_word_length = []
for file in os.listdir(text_dir):
    x, y = cleaned_words(file)
    word_count.append(x)
    average_word_length.append(y)

# To calculate Personal Pronouns mentioned in the text, we use regex to find
# the counts of the words - “I,” “we,” “my,” “ours,” and “us”. Special care is taken
# so that the country name US is not included in the list.
def count_personal_pronouns(file):
    with open(os.path.join(text_dir, file), 'r') as f:
        text = f.read()
        personal_pronouns = ["I", "we", "my", "ours", "us"]
        count = 0
        for pronoun in personal_pronouns:
            count += len(re.findall(r"\b" + pronoun + r"\b", text))  # \b is used to match word boundaries
    return count

pp_count = []
for file in os.listdir(text_dir):
    x = count_personal_pronouns(file)
    pp_count.append(x)

output_df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')

# URL_ID 44, 57, 144 do not exist, i.e., page does not exist, throws 404 error
# so we are going to drop these rows from the table
output_df.drop([44-37, 57-37, 144-37], axis=0, inplace=True)

# These are the required parameters
variables = [positive_score,
             negative_score,
             polarity_score,
             subjectivity_score,
             avg_sentence_length,
             Percentage_of_Complex_words,
             Fog_Index,
             avg_sentence_length,
             complex_word_count,
             word_count,
             avg_syllable_word_count,
             pp_count,
             average_word_length]

# write the values to the dataframe
for i, var in enumerate(variables):
    output_df.iloc[:, i+2] = var

# now save the dataframe to the disk
output_df.to_csv('/content/drive/MyDrive/20211030 Test Assignment/Output_Data.csv', index=False)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
import datetime

# Directories
text_dir = "/content/drive/MyDrive/20211030 Test Assignment"
stopwords_dir = "/content/drive/MyDrive/20211030 Test Assignment/StopWords"
sentiment_dir = "/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary"

# Load all stop words from the stopwords directory
stop_words = set()
for file in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, file), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(set(f.read().splitlines()))

# Load positive and negative words
pos = set()
neg = set()
for file in os.listdir(sentiment_dir):
    if file == 'positive-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            pos.update(f.read().splitlines())
    elif file == 'negative-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            neg.update(f.read().splitlines())

# Load the output structure
output_df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')

# Function to process each text file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize and filter stop words
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate scores
    pos_words = [word for word in filtered_words if word.lower() in pos]
    neg_words = [word for word in filtered_words if word.lower() in neg]
    positive_score = len(pos_words)
    negative_score = len(neg_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(filtered_words) + 0.000001)

    # Calculate readability metrics
    sentences = text.split('.')
    num_sentences = len(sentences)
    num_words = len([word for word in text.split() if word.lower() not in nltk_stopwords.words('english')])

    complex_words = [word for word in filtered_words if sum(1 for letter in word if letter.lower() in 'aeiou') > 2]
    avg_sentence_length = num_words / num_sentences
    percentage_complex_words = len(complex_words) / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Calculate syllable and word metrics
    syllable_count = sum(max(1, sum(1 for letter in word if letter.lower() in 'aeiou')) for word in filtered_words)
    avg_syllable_word_count = syllable_count / len(filtered_words)

    # Calculate word count and average word length
    cleaned_words = [word for word in re.sub(r'[^\w\s]', '', text).split() if word.lower() not in nltk_stopwords.words('english')]
    word_count = len(cleaned_words)
    avg_word_length = sum(len(word) for word in cleaned_words) / word_count

    # Count personal pronouns
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    pp_count = sum(len(re.findall(r"\b" + pronoun + r"\b", text, re.IGNORECASE)) for pronoun in personal_pronouns)

    return [positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_complex_words, fog_index,
            avg_sentence_length, len(complex_words), word_count,
            avg_syllable_word_count, pp_count, avg_word_length]

# Process all files
results = []
for index, row in output_df.iterrows():
    file_name = f"{row['URL_ID']}.txt"
    file_path = os.path.join(text_dir, file_name)
    if os.path.exists(file_path):
        results.append(process_file(file_path))
    else:
        results.append([None] * 13)  # Append None values if file doesn't exist

# Update the DataFrame
for i, col in enumerate(output_df.columns[2:]):
    output_df[col] = [row[i] if row else None for row in results]

# Save the results using a file stream
from io import StringIO

# Convert DataFrame to CSV string
csv_string = output_df.to_csv(index=False)

# Write the CSV string to a file in Google Drive
output_dir = '/content/drive/MyDrive/output_articles'
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'Output_Data_{timestamp}.csv'
output_path = os.path.join(output_dir, output_file)

# Ensure the directory exists
os.makedirs(output_dir, exist_ok=True)

try:
    # Write the file
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(csv_string)
    print(f"Processing complete. Results saved to {output_path}")
except IsADirectoryError:
    print(f"Error: {output_path} is a directory. Please choose a different file name.")
except PermissionError:
    print(f"Error: No permission to write to {output_path}. Please check your access rights.")
except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing complete. Results saved to /content/drive/MyDrive/output_articles/Output_Data_20240621_200106.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
import re
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
import datetime
import nltk

nltk.download('punkt')
nltk.download('stopwords')

# Directories
text_dir = "/content/drive/MyDrive/20211030 Test Assignment/TitleText"
stopwords_dir = "/content/drive/MyDrive/20211030 Test Assignment/StopWords"
sentiment_dir = "/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary"

# Load all stop words from the stopwords directory
stop_words = set()
for file in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, file), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(set(f.read().splitlines()))

# Load positive and negative words
pos = set()
neg = set()
for file in os.listdir(sentiment_dir):
    if file == 'positive-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            pos.update(f.read().splitlines())
    elif file == 'negative-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            neg.update(f.read().splitlines())

# Load the output structure
output_df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')

# Function to process each text file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize and filter stop words
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate scores
    pos_words = [word for word in filtered_words if word.lower() in pos]
    neg_words = [word for word in filtered_words if word.lower() in neg]
    positive_score = len(pos_words)
    negative_score = len(neg_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(filtered_words) + 0.000001)

    # Calculate readability metrics
    sentences = text.split('.')
    num_sentences = len(sentences)
    num_words = len([word for word in text.split() if word.lower() not in nltk_stopwords.words('english')])

    complex_words = [word for word in filtered_words if sum(1 for letter in word if letter.lower() in 'aeiou') > 2]
    avg_sentence_length = num_words / num_sentences
    percentage_complex_words = len(complex_words) / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Calculate syllable and word metrics
    syllable_count = sum(max(1, sum(1 for letter in word if letter.lower() in 'aeiou')) for word in filtered_words)
    avg_syllable_word_count = syllable_count / len(filtered_words)

    # Calculate word count and average word length
    cleaned_words = [word for word in re.sub(r'[^\w\s]', '', text).split() if word.lower() not in nltk_stopwords.words('english')]
    word_count = len(cleaned_words)
    avg_word_length = sum(len(word) for word in cleaned_words) / word_count

    # Count personal pronouns
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    pp_count = sum(len(re.findall(r"\b" + pronoun + r"\b", text, re.IGNORECASE)) for pronoun in personal_pronouns)

    return [positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_complex_words, fog_index,
            avg_sentence_length, len(complex_words), word_count,
            avg_syllable_word_count, pp_count, avg_word_length]

# Process all files
results = []
for index, row in output_df.iterrows():
    file_name = f"{row['URL_ID']}.txt"
    file_path = os.path.join(text_dir, file_name)
    if os.path.exists(file_path):
        results.append(process_file(file_path))
    else:
        results.append([None] * 13)  # Append None values if file doesn't exist

# Update the DataFrame
for i, col in enumerate(output_df.columns[2:]):
    output_df[col] = [row[i] if row else None for row in results]

# Save the results using a file stream
from io import StringIO

# Convert DataFrame to CSV string
csv_string = output_df.to_csv(index=False)

# Write the CSV string to a file in Google Drive
output_dir = '/content/drive/MyDrive/output_articles'
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file_path = os.path.join(output_dir, f'Output_Data_{timestamp}.csv')

with open(output_file_path, 'w') as f:
    f.write(csv_string)

print(f"Output saved to {output_file_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Output saved to /content/drive/MyDrive/output_articles/Output_Data_20240622_053920.csv


In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests

# Load input Excel file
input_df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Input.xlsx')

# Directory to save the extracted text files
extracted_text_dir = "/content/drive/MyDrive/output_articles"
os.makedirs(extracted_text_dir, exist_ok=True)

# Function to extract article text from a URL
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            title = soup.title.string if soup.title else ""
            paragraphs = soup.find_all('p')
            article_text = " ".join([p.get_text() for p in paragraphs])
            return title + "\n" + article_text
        else:
            return ""
    except Exception as e:
        print(f"Error extracting {url}: {e}")
        return ""

# Extract text for each URL and save to a text file
for index, row in input_df.iterrows():
    url_id = row['URL_ID']
    url = row['URL']
    article_text = extract_text_from_url(url)
    if article_text:
        file_path = os.path.join(extracted_text_dir, f"{url_id}.txt")
        with open(file_path, 'w', encoding='utf-8') as f:
            f.write(article_text)


In [ ]:
import os
import re
import pandas as pd
from nltk.corpus import stopwords as nltk_stopwords
from nltk.tokenize import word_tokenize
import datetime
import nltk

nltk.download('punkt')
nltk.download('stopwords')

# Directories
text_dir = "/content/drive/MyDrive/20211030 Test Assignment/TitleText"
stopwords_dir = "/content/drive/MyDrive/20211030 Test Assignment/StopWords"
sentiment_dir = "/content/drive/MyDrive/20211030 Test Assignment/MasterDictionary"

# Load all stop words from the stopwords directory
stop_words = set()
for file in os.listdir(stopwords_dir):
    with open(os.path.join(stopwords_dir, file), 'r', encoding='ISO-8859-1') as f:
        stop_words.update(set(f.read().splitlines()))

# Load positive and negative words
pos = set()
neg = set()
for file in os.listdir(sentiment_dir):
    if file == 'positive-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            pos.update(f.read().splitlines())
    elif file == 'negative-words.txt':
        with open(os.path.join(sentiment_dir, file), 'r', encoding='ISO-8859-1') as f:
            neg.update(f.read().splitlines())

# Load the output structure
output_df = pd.read_excel('/content/drive/MyDrive/20211030 Test Assignment/Output Data Structure.xlsx')

# Function to process each text file
def process_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()

    # Tokenize and filter stop words
    words = word_tokenize(text)
    filtered_words = [word for word in words if word.lower() not in stop_words]

    # Calculate scores
    pos_words = [word for word in filtered_words if word.lower() in pos]
    neg_words = [word for word in filtered_words if word.lower() in neg]
    positive_score = len(pos_words)
    negative_score = len(neg_words)
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(filtered_words) + 0.000001)

    # Calculate readability metrics
    sentences = text.split('.')
    num_sentences = len(sentences)
    num_words = len([word for word in text.split() if word.lower() not in nltk_stopwords.words('english')])

    complex_words = [word for word in filtered_words if sum(1 for letter in word if letter.lower() in 'aeiou') > 2]
    avg_sentence_length = num_words / num_sentences
    percentage_complex_words = len(complex_words) / num_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)

    # Calculate syllable and word metrics
    syllable_count = sum(max(1, sum(1 for letter in word if letter.lower() in 'aeiou')) for word in filtered_words)
    avg_syllable_word_count = syllable_count / len(filtered_words)

    # Calculate word count and average word length
    cleaned_words = [word for word in re.sub(r'[^\w\s]', '', text).split() if word.lower() not in nltk_stopwords.words('english')]
    word_count = len(cleaned_words)
    avg_word_length = sum(len(word) for word in cleaned_words) / word_count

    # Count personal pronouns
    personal_pronouns = ["I", "we", "my", "ours", "us"]
    pp_count = sum(len(re.findall(r"\b" + pronoun + r"\b", text, re.IGNORECASE)) for pronoun in personal_pronouns)

    return [positive_score, negative_score, polarity_score, subjectivity_score,
            avg_sentence_length, percentage_complex_words, fog_index,
            avg_sentence_length, len(complex_words), word_count,
            avg_syllable_word_count, pp_count, avg_word_length]

# Process all files
results = []
for index, row in output_df.iterrows():
    file_name = f"{row['URL_ID']}.txt"
    file_path = os.path.join(text_dir, file_name)
    if os.path.exists(file_path):
        results.append(process_file(file_path))
    else:
        results.append([None] * 13)  # Append None values if file doesn't exist

# Update the DataFrame by starting from the column after 'URL'
for i, col in enumerate(output_df.columns[2:]):
    output_df[col] = [row[i] if row else None for row in results]

# Save the results using a file stream
from io import StringIO

# Convert DataFrame to CSV string
csv_string = output_df.to_csv(index=False)

# Write the CSV string to a file in Google Drive
output_dir = '/content/drive/MyDrive/output_articles'
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
output_file_path = os.path.join(output_dir, f'Output_Data_{timestamp}.csv')

with open(output_file_path, 'w') as f:
    f.write(csv_string)

print(f"Output saved to {output_file_path}")


Output saved to /content/drive/MyDrive/output_articles/Output_Data_20240622_060414.csv


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
